In [1]:
import pandas as pd
import re
import string
from spacy.lang.de.stop_words import STOP_WORDS
# Spacy
import spacy

In [2]:
pd.set_option('display.max_colwidth', None)
nlp = spacy.load('de_core_news_md')

In [3]:
REVIEWS_FILE_PATH = 'data/reviews_merged.csv'
REVIEWS_CLEANED_FILE_PATH = 'data/reviews_merged_cleaned.csv'
REVIEWS_CLEANED_UNLABELED_FILE_PATH = 'data/reviews_merged_unlabeled_cleaned.csv'

df = pd.read_csv(REVIEWS_FILE_PATH, sep=';')

In [4]:
df.columns = ['caption',
              'food_positive', 'food_negative',
              'service_positive', 'service_negative',
              'ambient_positive', 'ambient_negative',
              'price_positive', 'price_negative',
              'waiting_positive', 'waiting_negative',
              'rating']

In [5]:
df.head(3)

,caption,food_positive,food_negative,service_positive,service_negative,ambient_positive,ambient_negative,price_positive,price_negative,waiting_positive,waiting_negative,rating
0,Mega cooles Ambiente und Konzept die Bürger sind auch geschmacklich sehr lecker. Man kann seiner Bürger individuelle auf seine Bedürfnisse anpassen ( Allergien/vegetarisch/Co) …,1,0,0,0,0,0,0,0,0,0,4.0
1,"Super sympathische Mitarbeiter, eine wirklich freundliche und gemütliche Atmosphäre und fantastisches Essen! Wir kommen ganz sicher öfter. …",1,0,1,0,1,0,0,0,0,0,5.0
2,"DAS nenne ich Burger! Nicht so ein Schicki-Micki möchte gerne Burgerladen in Dortmund! *hüstel* Ein absoluter Kult! Maestro del Tozzi - ein Gedicht! Wann immer ich in der Gegend bin, Space Burger oder Lupo. Beides einfach Kult! (Translated by Google) THAT'S what I call burgers! Not such a fancy mic would like a burger shop in Dortmund! *cough* An absolute cult! Maestro del Tozzi - a poem! Whenever I'm in the area, Space Burger or Lupo. Both simply cult!",1,0,0,0,1,0,0,0,0,0,5.0


In [6]:
# Remove NaN caption
USEFUL_COLUMNS = ['caption',
                  'food_positive', 'service_positive', 'ambient_positive', 'price_positive', 'waiting_positive', 'rating']
df = df.loc[~df['caption'].isnull(), USEFUL_COLUMNS]
df.reset_index(drop=True)
df.head(3)

,caption,food_positive,service_positive,ambient_positive,price_positive,waiting_positive,rating
0,Mega cooles Ambiente und Konzept die Bürger sind auch geschmacklich sehr lecker. Man kann seiner Bürger individuelle auf seine Bedürfnisse anpassen ( Allergien/vegetarisch/Co) …,1,0,0,0,0,4.0
1,"Super sympathische Mitarbeiter, eine wirklich freundliche und gemütliche Atmosphäre und fantastisches Essen! Wir kommen ganz sicher öfter. …",1,1,1,0,0,5.0
2,"DAS nenne ich Burger! Nicht so ein Schicki-Micki möchte gerne Burgerladen in Dortmund! *hüstel* Ein absoluter Kult! Maestro del Tozzi - ein Gedicht! Wann immer ich in der Gegend bin, Space Burger oder Lupo. Beides einfach Kult! (Translated by Google) THAT'S what I call burgers! Not such a fancy mic would like a burger shop in Dortmund! *cough* An absolute cult! Maestro del Tozzi - a poem! Whenever I'm in the area, Space Burger or Lupo. Both simply cult!",1,0,1,0,0,5.0


In [7]:
# Remove the section translated by google if present
df['caption'] = df['caption'].str.split('(Translated by Google)').str[0]
df.head(3)

,caption,food_positive,service_positive,ambient_positive,price_positive,waiting_positive,rating
0,Mega cooles Ambiente und Konzept die Bürger sind auch geschmacklich sehr lecker. Man kann seiner Bürger individuelle auf seine Bedürfnisse anpassen ( Allergien/vegetarisch/Co) …,1,0,0,0,0,4.0
1,"Super sympathische Mitarbeiter, eine wirklich freundliche und gemütliche Atmosphäre und fantastisches Essen! Wir kommen ganz sicher öfter. …",1,1,1,0,0,5.0
2,"DAS nenne ich Burger! Nicht so ein Schicki-Micki möchte gerne Burgerladen in Dortmund! *hüstel* Ein absoluter Kult! Maestro del Tozzi - ein Gedicht! Wann immer ich in der Gegend bin, Space Burger oder Lupo. Beides einfach Kult! (",1,0,1,0,0,5.0


In [8]:
print (STOP_WORDS)

{'deine', 'werde', 'vielem', 'infolgedessen', 'dem', 'na', 'davon', 'hätten', 'en', 'uhr', 'wurde', 'sechster', 'ehrlich', 'siebentes', 'welches', 'wann', 'tat', 'durften', 'dies', 'genug', 'hinter', 'drin', 'satt', 'unsere', 'damit', 'mögt', 'bereits', 'dann', 'übrigens', 'ausser', 'viele', 'denn', 'mir', 'neuntes', 'während', 'a', 'einiger', 'indem', 'keine', 'außerdem', 'jahre', 'dank', 'am', 'nun', 'gewollt', 'mancher', 'anderem', 'meinem', 'daran', 'jene', 'ebenso', 'keiner', 'drittes', 'dort', 'aus', 'heisst', 'jedem', 'an', 'ach', 'daraus', 'macht', 'kein', 'wie', 'demgegenüber', 'neun', 'gekonnt', 'also', 'zu', 'sein', 'dahinter', 'beide', 'wird', 'weiter', 'ihm', 'beispiel', 'ersten', 'ganze', 'recht', 'niemand', 'lang', 'hoch', 'rechte', 'konnten', 'darfst', 'dafür', 'deinem', 'achte', 'fünfter', 'siebten', 'darauf', 'können', 'wahr', 'wurden', 'zurück', 'den', 'achten', 'meiner', 'deiner', 'würde', 'würden', 'wem', 'kleinen', 'rechter', 'fünfte', 'einem', 'außer', 'ich', 'ac

In [9]:
REGX_USERNAME = r"@[A-Za-z0-9$-_@.&+]+"
REGX_URL = r"https?://[A-Za-z0-9./]+"
def preprocessing(text):
    text = text.lower()

    # Remove
    text = re.sub(REGX_USERNAME, ' ', text)
    text = re.sub(REGX_URL, ' ', text)

    # Replace Emojis
    emojis = {
        ':)': 'positive emotionen',
        ':(': 'negative emotionen'
    }

    for e in emojis:
        text = text.replace(e, emojis[e])

    tokens = [token.text for token in nlp(text)]

    # Remove Stop Words, punctuations and words with less than 3 characters
    tokens = [t for t in tokens if
              t not in STOP_WORDS and
              t not in string.punctuation and
              len(t) > 3]

    tokens = [t for t in tokens if not t.isdigit()]

    text = " ".join(tokens);

    # Lemmatization
    '''
    allowed_postags = ["NOUN", "ADJ", "VERB", "ADV"]
    doc = nlp(text)
    new_text = []
    for token in doc:
        if token.pos_ in allowed_postags:
            new_text.append(token.lemma_)
    lemmatized = " ".join(new_text).lower()
     '''

    return  text #lemmatized

df["caption_clean"] = df["caption"].apply(preprocessing)

In [10]:
df.head(3)

,caption,food_positive,service_positive,ambient_positive,price_positive,waiting_positive,rating,caption_clean
0,Mega cooles Ambiente und Konzept die Bürger sind auch geschmacklich sehr lecker. Man kann seiner Bürger individuelle auf seine Bedürfnisse anpassen ( Allergien/vegetarisch/Co) …,1,0,0,0,0,4.0,mega cooles ambiente konzept bürger geschmacklich lecker bürger individuelle bedürfnisse anpassen allergien vegetarisch
1,"Super sympathische Mitarbeiter, eine wirklich freundliche und gemütliche Atmosphäre und fantastisches Essen! Wir kommen ganz sicher öfter. …",1,1,1,0,0,5.0,super sympathische mitarbeiter freundliche gemütliche atmosphäre fantastisches essen sicher öfter
2,"DAS nenne ich Burger! Nicht so ein Schicki-Micki möchte gerne Burgerladen in Dortmund! *hüstel* Ein absoluter Kult! Maestro del Tozzi - ein Gedicht! Wann immer ich in der Gegend bin, Space Burger oder Lupo. Beides einfach Kult! (",1,0,1,0,0,5.0,nenne burger schicki-micki gerne burgerladen dortmund hüstel absoluter kult maestro tozzi gedicht gegend space burger lupo beides einfach kult


In [11]:
# renaming dataset columns, grabbing only the cleaned captions
USEFUL_COLUMNS_AND_CLEAN = USEFUL_COLUMNS[:]
USEFUL_COLUMNS_AND_CLEAN.remove('caption') # don't select caption at this point
USEFUL_COLUMNS_AND_CLEAN.append('caption_clean') # Select caption_clean

df = df[USEFUL_COLUMNS_AND_CLEAN]

df = df.rename(columns={"caption_clean": "caption"}) #
USEFUL_COLUMNS_AND_CLEAN.append('caption') # add caption
USEFUL_COLUMNS_AND_CLEAN.remove('caption_clean') # Remove caption_clean
df.reset_index(drop=True)

,food_positive,service_positive,ambient_positive,price_positive,waiting_positive,rating,caption
0,1,0,0,0,0,4.0,mega cooles ambiente konzept bürger geschmacklich lecker bürger individuelle bedürfnisse anpassen allergien vegetarisch
1,1,1,1,0,0,5.0,super sympathische mitarbeiter freundliche gemütliche atmosphäre fantastisches essen sicher öfter
2,1,0,1,0,0,5.0,nenne burger schicki-micki gerne burgerladen dortmund hüstel absoluter kult maestro tozzi gedicht gegend space burger lupo beides einfach kult
3,0,0,0,0,0,1.0,gestern dritt selben burger bestellt gestern abend magen darm burger lecker irgendetwas schmeckte komisch erwartet
4,1,0,1,0,0,4.0,fanden burger restaurant klasse individuelle möglichkeiten zudem fleischlose alternativen ambiente zeitsprung vergangenheit kartenzahlung heutzutage zeitgemäß
...,...,...,...,...,...,...,...
354,1,0,0,0,0,5.0,hervorragende neapolitanische pizza
355,1,1,1,0,1,5.0,staff super fruendliche atmosphäre chillig schöne auswahl pizzas lecker schnell fertig wartezeit qalität leidet
356,1,0,0,0,0,5.0,leckere pizza hochpreisig
357,1,0,0,0,0,5.0,amazing pizza


In [12]:
USEFUL_COLUMNS_AND_CLEAN

['food_positive',
 'service_positive',
 'ambient_positive',
 'price_positive',
 'waiting_positive',
 'rating',
 'caption']

In [13]:
# Remove null & empty values once again for 'caption_clean' column
df = df.loc[~df['caption'].isnull(), USEFUL_COLUMNS_AND_CLEAN]
df = df[df['caption'].str.strip().astype(bool)]

In [14]:
df.head(3)

,food_positive,service_positive,ambient_positive,price_positive,waiting_positive,rating,caption
0,1,0,0,0,0,4.0,mega cooles ambiente konzept bürger geschmacklich lecker bürger individuelle bedürfnisse anpassen allergien vegetarisch
1,1,1,1,0,0,5.0,super sympathische mitarbeiter freundliche gemütliche atmosphäre fantastisches essen sicher öfter
2,1,0,1,0,0,5.0,nenne burger schicki-micki gerne burgerladen dortmund hüstel absoluter kult maestro tozzi gedicht gegend space burger lupo beides einfach kult


In [15]:
df[['caption', 'rating']].to_csv(REVIEWS_CLEANED_UNLABELED_FILE_PATH, sep=';')
df.to_csv(REVIEWS_CLEANED_FILE_PATH, sep=';')